# Introduction to Visualization

In the previous step of the tutorial we made some plots using `hvplot`, which provides interactive plots using a syntax similar to the Pandas `.plot()` method. In addition to Pandas, `hvPlot` supports XArray, Dask, GeoPandas, and a variety of other libraries. The result of an `hvplot` call is a `holoviews` object. This might seem a little mysterious, so we'll take a minute to show how this works. In this tutorial step we'll use generated data for simplicity.

In [1]:
import numpy as np
import pandas as pd

index = pd.date_range('1/1/2000', periods=1000)

np.random.seed(11)  # seeding the state to make the output identical each time
data = np.random.randn(1000, 4)

df = pd.DataFrame(data, index=index, columns=list('ABCD')).cumsum()

df.head()

,A,B,C,D
2000-01-01,1.749455,-0.286073,-0.484565,-2.653319
2000-01-02,1.741170,-0.605704,-1.021194,-2.337916
2000-01-03,2.162221,-1.671307,-1.907434,-2.813649
2000-01-04,2.851903,-1.110115,-3.212983,-3.933125
2000-01-05,3.588741,0.464519,-3.244058,-4.616571


Use `hvplot` to product an interactively explorable plot with panning, zooming, hovering, and clickable/selectable legends:

In [2]:
import hvplot.pandas

df.hvplot()

/home/hayley/miniconda3/envs/earthml/lib/python3.7/site-packages/bokeh/plotting/helpers.py:21: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, OrderedDict, Sequence
/home/hayley/miniconda3/envs/earthml/lib/python3.7/site-packages/jinja2/utils.py:485: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping


/home/hayley/miniconda3/envs/earthml/lib/python3.7/site-packages/bokeh/core/property/bases.py:237: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  return new == old
/home/hayley/miniconda3/envs/earthml/lib/python3.7/site-packages/bokeh/core/property/container.py:102: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(value, (collections.Container, collections.Sized, collections.Iterable))


:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [26]:
# Exercise: Scroll in and out of the plot, click on an item in the legend, save a png

In [27]:
I
import numpy as np
X = np.arange(-5, 5, 0.25)
Y = np.arange(-5, 5, 0.25)
X, Y = np.meshgrid(X, Y)
R = np.sqrt(X**2 + Y**2)
Z = np.sin(R)
img = hv.Image(Z)

img

NameError: name 'hv' is not defined

## Selection

To inspect this plot more closely we can assign it to a variable and print that variable. We'll see that we in fact have a holoviews object of type `NdOverlay` and subtype `Curve`. 

In [3]:
plot = df.hvplot()
print(plot)

:NdOverlay   [Variable]
   :Curve   [index]   (value)


But what does that mean? What we got out of hvplot was actually a mapping. 

In [9]:
# Exercise: Use plot.keys() to inspect the keys of the mapping
plot.keys()
plot.Curve.kdims


AttributeError: 'NdOverlay' object has no attribute 'Curve'

We can select just one of the variables from the mapping just like you would from a dictionary:

In [ ]:
plot['A']

In [ ]:
# Exercise: select a different label from the plot.

We can create an overlay of just a few of the plots using the `*` operator

In [19]:
p2 = plot['B'] * plot['C']
print(p2.__class__.mro())

[<class 'holoviews.core.overlay.Overlay'>, <class 'holoviews.core.dimension.ViewableTree'>, <class 'holoviews.core.tree.AttrTree'>, <class 'holoviews.core.overlay.CompositeOverlay'>, <class 'holoviews.core.dimension.ViewableElement'>, <class 'holoviews.core.dimension.Dimensioned'>, <class 'holoviews.core.dimension.LabelledData'>, <class 'param.parameterized.Parameterized'>, <class 'holoviews.core.layout.Composable'>, <class 'object'>]


## Subplots

We can also switch it around so that instead of having all the plots overlayed, we can display them next to each other. For this we will use the '+' operator. Note that we also specified that there should only be one column, otherwise they would be added row-wise. 

In [20]:
(plot['B'] + plot['C']).cols(1)

/home/hayley/miniconda3/envs/earthml/lib/python3.7/site-packages/bokeh/core/property/bases.py:237: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  return new == old


:Layout
   .Curve.I  :Curve   [index]   (value)
   .Curve.II :Curve   [index]   (value)

Another way to achieve a similar result is to use the `subplots` keyword argument in `hvplot`. 

In [21]:
df.hvplot(subplots=True, width=300).cols(2)

/home/hayley/miniconda3/envs/earthml/lib/python3.7/site-packages/bokeh/core/property/bases.py:237: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  return new == old


:NdLayout   [Variable]
   :Curve   [index]   (value)

In [ ]:
# Exercise: Try to create something other than a line plot. Perhaps a scatter - df.hvplot.scatter(). 
# Challenge: Use tab completion to explore other plot types.

# Visualizing images

In the [Data Ingestion](01_Data_Ingestion.ipynb) tutorial, we saw how you can view Landsat imagery loaded with intake. First let us import `intake` and `hvplot.xarray` to load `hvplot` support for `xarray`:

In [22]:
import intake
import hvplot.xarray

/home/hayley/miniconda3/envs/earthml/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


Now we can open the catalog and read in our data for Landsat 5 and 8. We'll be using the small versions in this tutorial, but feel free to change them to `cat.landsat_5` and `cat.landsat_8` to use the large versions instead.

In [24]:
cat = intake.open_catalog('../catalog.yml')
landsat_5 = cat.landsat_5_small.read_chunked()
landsat_8 = cat.landsat_8_small.read_chunked()

/home/hayley/miniconda3/envs/earthml/lib/python3.7/site-packages/intake/catalog/local.py:566: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text)


Using `geo=True` in `hvplot`, we can ensure the appropriate coordinate reference system is used. In the next cell, we use this along with `cmap='fire'` to change the color map:

In [25]:
landsat_5.hvplot(x='x', y='y', rasterize=True, geo=True, width=400, cmap='fire')

:DynamicMap   [band]
   :Image   [x,y]   (value)

In [ ]:
# Exercise: Try adjusting the width and adding a height option to make the overall plot bigger or smaller.

The 'fire' colormap is one of the [*perceptually uniform colormaps*](https://arxiv.org/abs/1509.03700) provided by [colorcet](http://colorcet.pyviz.org/). These colormaps are especially designed to help reveal features of the data that may be lost when using colormaps that are not perceptually uniform.

## Combining images with `+`

In the same way you can combine curves with `+` (as shown above using data loaded from a pandas DataFrame), you can do the same using images created from `xarray` objects.

In the next cell, we use different colormaps for the  Landsat 5 and Landsat 8 data, datashade them and position them next to each other:

In [ ]:
landsat_5_plot = landsat_5.hvplot(x='x', y='y', width=300, datashade=True, geo=True, cmap='Reds')
landsat_8_plot = landsat_8.hvplot(x='x', y='y', width=300, datashade=True, geo=True, cmap='Blues')
landsat_5_plot + landsat_8_plot

As `datashade=True`, the plots refresh with the available data when the zoom tool is used. Note that the plots have linked axes making comparison between plots easier. Note that the plots will only update to show higher resolution if a live Python process is running; on a web site zooming will only ever show the original set of pixels, getting larger on a zoom rather than revealing more data as you'll see if Python is available.

hvPlot also supports a large range of plot types and options not shown here. A more detailed description of visualization options will follow in the [Data Visualization](05_Data_Visualization.ipynb) tutorial. See the [hvplot.pyviz.org](https://hvplot.pyviz.org) website for further details, and [holoviews.org](http://holoviews.org), [geoviews.org](http://geoviews.org), and [datashader.org](http://datashader.org) for information about the underlying functionality and advanced usage. You can also work through the extensive tutorials at [pyviz.org](http://pyviz.org)